The Iowa Democratic Party is uploading the results of Tuesday's caucus to https://results.thecaucuses.org.
I haven't found a source for the raw data, so we have to scrape it.

Unfortunatley, the HTML isn't the cleanest... (https://twitter.com/therriaultphd/status/1224833286599585793).

In [1]:
import io
import requests
import lxml.html
import pandas as pd

url = "https://results.thecaucuses.org"
r = requests.get(url)

root = lxml.html.parse(io.StringIO(r.text)).getroot()

In [2]:
# Bennet, Biden, etc.
head = root.find_class("thead")[0]
header = [x.text for x in list(head.iterchildren())]

# First Expression, Final Expression, SDE, ...
subhead = root.find_class("sub-head")[0]
subheader = [x.text for x in list(subhead.iterchildren())]

In [3]:
columns = pd.MultiIndex.from_arrays([
    pd.Series(header).fillna(method='ffill'),
    pd.Series(subheader).fillna(method='ffill').fillna('')
], names=['candidate', 'round'])

In [4]:
counties = root.find_class("precinct-county")
county_names = [x[0].text for x in counties]
counties_data = root.find_class("precinct-data")
county = counties_data[0]
rows = []

In [5]:
for name, county in zip(county_names, counties_data):
    if len(county) > 1:
        # satellites only have a total
        county = county[:-1]

    for precinct in county:
        # exclude total
        rows.append((name,) + tuple(x.text for x in precinct))

In [6]:
df = (
    pd.DataFrame(rows, columns=columns)
      .set_index(['County', 'Precinct'])
      .stack(level='candidate')
      .apply(pd.to_numeric)
)
df

round                                Final Expression  First Expression  \
County        Precinct  candidate                                         
Adair         1NW ADAIR Bennet                      0                 0   
                        Biden                       6                 6   
                        Bloomberg                   0                 0   
                        Buttigieg                   8                 8   
                        Delaney                     0                 0   
...                                               ...               ...   
CD4 Satellite Total     Sanders                     0                 0   
                        Steyer                      0                 0   
                        Uncommitted                 0                 0   
                        Warren                      0                 0   
                        Yang                        0                 0   

round                                   SDE  
County        Precinct  candidate            
Adair         1NW ADAIR Bennet       0.0000  
                        Biden        0.0784  
                        Bloomberg    0.0000  
                        Buttigieg    0.0784  
                        Delaney      0.0000  
...                                     ...  
CD4 Satellite Total     Sanders      0.0000  
                        Steyer       0.0000  
                        Uncommitted  0.0000  
                        Warren       0.0000  
                        Yang         0.0000  

[15456 rows x 3 columns]